In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim

transform = transforms.Compose(
    [
        transforms.Resize(224),  # VGG 输入是 224×224
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        # VGG16 卷积部分
        self.features = nn.Sequential(
            # conv1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # conv2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # conv3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # conv4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # conv5
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # 分类头
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),  # 输入尺寸 224x224 时
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # flatten除了 batch 维度
        x = self.classifier(x)
        return x

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
# 随机初始化（不加载预训练权重）
vgg_scratch = models.vgg16(pretrained=False)
vgg_scratch.classifier[6] = torch.nn.Linear(4096, 10)  # CIFAR10 10类
vgg_pretrained = models.vgg16(pretrained=True)
vgg_pretrained.classifier[6] = torch.nn.Linear(4096, 10)

# 冻结卷积层，只训练分类头
for param in vgg_pretrained.features.parameters():
    param.requires_grad = False

def train(model, trainloader, criterion, optimizer, device, epochs=5):
    model.to(device)
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(trainloader):.4f}")

def test(model, testloader, device):
    model.to(device)
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = correct / total
    print(f"Accuracy: {acc:.4f}")
    return acc

device = torch.device("cuda")

criterion = nn.CrossEntropyLoss()

# 自训练模型
optimizer_scratch = optim.Adam(vgg_scratch.parameters(), lr=1e-4)
train(vgg_scratch, trainloader, criterion, optimizer_scratch, device, epochs=5)
acc_scratch = test(vgg_scratch, testloader, device)

# 预训练模型
optimizer_pretrained = optim.Adam(filter(lambda p: p.requires_grad, vgg_pretrained.parameters()), lr=1e-4)
train(vgg_pretrained, trainloader, criterion, optimizer_pretrained, device, epochs=5)
acc_pretrained = test(vgg_pretrained, testloader, device)

print(f"From scratch acc: {acc_scratch:.4f}")
print(f"Pretrained acc: {acc_pretrained:.4f}")

In [1]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.6
